In [1]:
cd .. 

/home/admin-hieunn/important/VNUIS-Chatbot


/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import numpy as np
import json
import torch
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
import pandas as pd
from datasets import Dataset
from huggingface_hub import login
import os
from dotenv import load_dotenv
from datetime import datetime
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer
from dotenv import load_dotenv

load_dotenv()
corpus = pd.read_csv("./Data/finetune_embedding/corpus.csv")
queries = pd.read_csv("./Data/finetune_embedding/queries.csv")
relevant_docs_data = pd.read_csv("./Data/finetune_embedding/eval.csv")


corpus = Dataset.from_pandas(corpus)
queries = Dataset.from_pandas(queries)
relevant_docs_data = Dataset.from_pandas(relevant_docs_data)
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus["relevantdocs_id"], corpus["Relevant docs"])
)  # Our corpus (cid => document)
queries = dict(
    zip(queries["queries_id"], queries["Question"])
)  # Our queries (qid => question)
# Convert integer keys to string keys
queries = {str(key): value for key, value in queries.items()}
# Now your 'queries' dictionary will have string keys

relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for qid, corpus_ids in zip(
    relevant_docs_data["queries_id"], relevant_docs_data["relevantdocs_id"]
):
    qid = str(qid)
    corpus_ids = str(corpus_ids)
    if qid not in relevant_docs:
        relevant_docs[qid] = set()
    relevant_docs[qid].add(corpus_ids)

model = SentenceTransformer(
    # model_name_or_path="BAAI/bge-m3",
    model_name_or_path="/home/admin-hieunn/important/VNUIS-Chatbot/Model/vnuis_embedding_bge_20241227",
    trust_remote_code=True,
)
matryoshka_dimensions = [768, 512, 256, 128, 64]  # Important: large to small
matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

# Evaluate the model and ensure it's all on the correct device
results = evaluator(model)

for k, v in results.items():
    print(k, v)

/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


dim_768_cosine_accuracy@1 0.6781059947871416
dim_768_cosine_accuracy@3 0.8783666377063423
dim_768_cosine_accuracy@5 0.9135534317984362
dim_768_cosine_accuracy@10 0.94874022589053
dim_768_cosine_precision@1 0.6781059947871416
dim_768_cosine_precision@3 0.29278887923544744
dim_768_cosine_precision@5 0.18271068635968718
dim_768_cosine_precision@10 0.09487402258905299
dim_768_cosine_recall@1 0.6781059947871416
dim_768_cosine_recall@3 0.8783666377063423
dim_768_cosine_recall@5 0.9135534317984362
dim_768_cosine_recall@10 0.94874022589053
dim_768_cosine_ndcg@10 0.8248530495073324
dim_768_cosine_mrr@10 0.7839509812033691
dim_768_cosine_map@100 0.7867942877699591
dim_512_cosine_accuracy@1 0.6772371850564727
dim_512_cosine_accuracy@3 0.8718505647263249
dim_512_cosine_accuracy@5 0.9122502172024327
dim_512_cosine_accuracy@10 0.9539530842745438
dim_512_cosine_precision@1 0.6772371850564727
dim_512_cosine_precision@3 0.290616854908775
dim_512_cosine_precision@5 0.18245004344048651
dim_512_cosine_pre

In [3]:
def prepare_training_dataset(queries, corpus, relevant_docs):
    anchors = []
    positives = []
    for query_id, docs in relevant_docs.items():
        for doc_id in docs:
            anchors.append(queries[query_id])
            positives.append(corpus[doc_id])
    df = {"anchor": anchors, "positive": positives}

    return Dataset.from_dict(df)


pairs = prepare_training_dataset(queries, corpus, relevant_docs)

matryoshka_dimensions = [768, 512, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

# Get today's date in the desired format (e.g., YYYYMMDD)
today_date = datetime.today().strftime("%Y%m%d")

# Define the model name with today's date
model_name = f"vnuis_embedding_bge_{today_date}"

args = SentenceTransformerTrainingArguments(
    output_dir=f"./Model/{model_name}",  # output directory and hugging face model ID
    num_train_epochs=30,  # number of epochs
    per_device_train_batch_size=4,  # train batch size
    gradient_accumulation_steps=128,  # for a global batch size of 512
    per_device_eval_batch_size=4,  # evaluation batch size
    gradient_checkpointing=True,
    warmup_ratio=0.1,  # warmup ratio
    learning_rate=2e-5,  # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",  # use constant learning rate scheduler
    optim="adamw_torch_fused",  # use fused adamw optimizer
    # tf32=False,                                  # use tf32 precision
    bf16=True,  # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="steps",  # evaluate after each epoch
    save_strategy="steps",  # save after each epoch
    logging_steps=10,  # log every 10 steps
    save_steps = 10,
    save_total_limit=2,  # save only the last 3 models
    load_best_model_at_end=True,  # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,  # training arguments
    train_dataset=pairs,
    loss=train_loss,
    evaluator=evaluator,
)
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save the best model
trainer.save_model()


[2024-12-27 16:31:24,205] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
  8%|▊         | 10/120 [05:42<1:01:10, 33.37s/it]

{'loss': 2.6823, 'grad_norm': 17.194053649902344, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.22}


  8%|▊         | 10/120 [06:24<1:01:10, 33.37s/it]

{'eval_dim_768_cosine_accuracy@1': 0.7072111207645526, 'eval_dim_768_cosine_accuracy@3': 0.8801042571676803, 'eval_dim_768_cosine_accuracy@5': 0.9139878366637706, 'eval_dim_768_cosine_accuracy@10': 0.946133796698523, 'eval_dim_768_cosine_precision@1': 0.7072111207645526, 'eval_dim_768_cosine_precision@3': 0.29336808572256007, 'eval_dim_768_cosine_precision@5': 0.1827975673327541, 'eval_dim_768_cosine_precision@10': 0.09461337966985228, 'eval_dim_768_cosine_recall@1': 0.7072111207645526, 'eval_dim_768_cosine_recall@3': 0.8801042571676803, 'eval_dim_768_cosine_recall@5': 0.9139878366637706, 'eval_dim_768_cosine_recall@10': 0.946133796698523, 'eval_dim_768_cosine_ndcg@10': 0.8356623746856879, 'eval_dim_768_cosine_mrr@10': 0.7993144332740328, 'eval_dim_768_cosine_map@100': 0.8013748706823961, 'eval_dim_512_cosine_accuracy@1': 0.6980886185925282, 'eval_dim_512_cosine_accuracy@3': 0.8748913987836664, 'eval_dim_512_cosine_accuracy@5': 0.9152910512597741, 'eval_dim_512_cosine_accuracy@10': 0.9

...

{'loss': 0.0463, 'grad_norm': 1.7233991622924805, 'learning_rate': 0.0, 'epoch': 26.67}


100%|██████████| 120/120 [1:17:23<00:00, 35.46s/it]

{'eval_dim_768_cosine_accuracy@1': 0.6416159860990444, 'eval_dim_768_cosine_accuracy@3': 0.8284100781928757, 'eval_dim_768_cosine_accuracy@5': 0.8844483058210252, 'eval_dim_768_cosine_accuracy@10': 0.944396177237185, 'eval_dim_768_cosine_precision@1': 0.6416159860990444, 'eval_dim_768_cosine_precision@3': 0.2761366927309586, 'eval_dim_768_cosine_precision@5': 0.17688966116420504, 'eval_dim_768_cosine_precision@10': 0.09443961772371849, 'eval_dim_768_cosine_recall@1': 0.6416159860990444, 'eval_dim_768_cosine_recall@3': 0.8284100781928757, 'eval_dim_768_cosine_recall@5': 0.8844483058210252, 'eval_dim_768_cosine_recall@10': 0.944396177237185, 'eval_dim_768_cosine_ndcg@10': 0.7955669732681404, 'eval_dim_768_cosine_mrr@10': 0.7476478355605195, 'eval_dim_768_cosine_map@100': 0.7514706688798185, 'eval_dim_512_cosine_accuracy@1': 0.6463944396177237, 'eval_dim_512_cosine_accuracy@3': 0.8301476976542137, 'eval_dim_512_cosine_accuracy@5': 0.8805386620330148, 'eval_dim_512_cosine_accuracy@10': 0.9

100%|██████████| 120/120 [1:17:29<00:00, 38.74s/it]


{'train_runtime': 4649.2264, 'train_samples_per_second': 14.854, 'train_steps_per_second': 0.026, 'train_loss': 0.43819571236769356, 'epoch': 26.67}


In [4]:

fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluate the model
results = evaluator(fine_tuned_model)

for k, v in results.items():
    print(k, v)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.47 GiB. GPU 0 has a total capacity of 15.70 GiB of which 1.01 GiB is free. Including non-PyTorch memory, this process has 14.67 GiB memory in use. Of the allocated memory 12.12 GiB is allocated by PyTorch, and 2.39 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)